# Problem 2 #

In [85]:
import numpy as np 
from collections import Counter
import sys
from sympy import *
np.set_printoptions(threshold=sys.maxsize)

In [134]:
N = 10 
r = 0.02 
d = 0.9
S0 = 100
dt = 1/N
K = 100

#### Lets populate the stock price array ####

In [124]:
stockPrice = np.zeros([2**N,N+1],dtype = 'longdouble')
stockPrice[0,0] = S0

for column in range(N):
    
    for row in range((Counter(stockPrice[:,column] > 0)[True]*2)):
        
        try:
            
            u = 1.1 * (column+16)/(column+17)
            d = 0.9
        
            stockPrice[row*2,column+1] = stockPrice[row,column]*u
            stockPrice[(row*2)+1,column+1] = stockPrice[row,column]*d
        
        except:
            break

#### Lets populate the Risk Neutral Probabilities array ####

In [125]:
Q = np.zeros([2**N,N],dtype = 'longdouble')
Q.shape

rho = 1 + r*dt

In [126]:
for column in range(Q.shape[1]):
    
    colIndex = Q.shape[1] - column - 1
    for row in range(int(Counter(stockPrice[:,colIndex+1] >0)[True]/2)):
        
        x = Symbol("x")
        q = solve([stockPrice[2*row,colIndex+1]*x/rho + (1-x)*stockPrice[2*row+1,colIndex+1]/rho - stockPrice[row,colIndex]],x)[x]
        
        Q[2*row,colIndex] = q
        Q[(2*row)+1,colIndex] = 1-q
        
        

#### Lets now Calculate the Price of the European Put ####

In [190]:
putPrice = np.zeros((stockPrice.shape),dtype = 'longdouble')

for i in range(len(putPrice)):
    putPrice[i,-1] = max(K-stockPrice[i,-1],0)

In [191]:
for column in range(putPrice.shape[1]-1):
    
    colIndex = putPrice.shape[1] - 2 - column
    for row in range(int(len(putPrice)/(2**(column+1)))):
        
        putPrice[row,colIndex] = Q[row,colIndex]*putPrice[row,colIndex+1]/rho + Q[row+1,colIndex]*putPrice[row+1,colIndex+1]/rho

In [194]:
putPrice

array([[5.51570223e-02, 3.35146256e-02, 1.53003424e-02, 3.97792173e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.21909033e-01, 8.41305849e-02, 4.42051629e-02, 1.34407499e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.35903823e-01, 1.00063235e-01, 5.73001556e-02, 1.94909699e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.27955590e-01, 2.28020542e-01, 2.02176084e-01, 1.42208858e-01, 6.10310994e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.28565221e-01, 2.39564049e-01, 2.29581357e-01, 1.80828417e-01, 9.12344696e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.73553870e-01, 2.0